In [ ]:
import astropy.units as u
import numpy as np
import os
from astropy.table import Table

os.environ["PIXEDFIT_HOME"] = "/nvme/scratch/work/tharvey/piXedfit/"
from EXPANSE import ResolvedGalaxy, ResolvedGalaxies
from matplotlib import pyplot as plt
import matplotlib.patheffects as pe
import glob
from scipy import signal
from scipy.interpolate import interp1d
from scipy.stats import binned_statistic
# Change dpi to make plots larger

plt.rcParams["figure.dpi"] = 100

# Disable tex in matplotlib

plt.rcParams["text.usetex"] = False
%matplotlib inline

In [ ]:
galaxies = ResolvedGalaxies(
    ResolvedGalaxy.init_all_field_from_h5("JOF_psfmatched", n_jobs = 6)
)

In [ ]:
single_galaxies = ResolvedGalaxies(ResolvedGalaxy.init_all_field_from_h5("JOF_psfmatched", n_jobs = 6, h5_folder='/nvme/scratch/work/tharvey/EXPANSE/galaxies/singlebin/'))

In [ ]:
galaxy = ResolvedGalaxy.init_from_h5("JOF_psfmatched_15021.h5")

'''
galaxy.add_det_galaxy_region(overwrite=False)

galaxy.pixedfit_processing(gal_region_use="detection")

galaxy.pixedfit_binning(animate=True, overwrite=True, 
                        animation_save_path= f'/nvme/scratch/work/tharvey/EXPANSE/galaxies/diagnostic_plots/{galaxy.galaxy_id}_pixedfit_binning.gif')


galaxy.voronoi_binning(SNR_reqs=7, galaxy_region='detection', overwrite=False,
                               use_only_widebands=False, plot=True, quiet=False,
                               ref_band = 'combined_average')

'''

In [ ]:
65*0.03

In [ ]:
#galaxy.run_galfitm()
#galaxy.add_psf_models('/nvme/scratch/work/tharvey/EXPANSE/psfs/psf_models/star_stack/JOF_psfmatched/', file_ending='_psf.fits')
galaxy.run_autogalaxy(model_type='sersic', band='F444W', mask_type='circular', mask_radius=0.95)

In [ ]:
for galaxy in galaxies_not_outshined:
    if 'pixedfit_nomin' not in galaxy.photometry_table['star_stack'].keys():
        print(galaxy.galaxy_id)
        print(galaxy.photometry_table.keys())
        galaxy.measure_flux_in_bins(binmap_type='pixedfit_nomin')

In [ ]:
#table = galaxies.save_to_fits(overwrite=False, save=False)

table = Table.read('/nvme/scratch/work/tharvey/EXPANSE/scripts/galaxies.fits')


## Resolved Mass vs. Integrated Mass

In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_lognorm", "photoz_delayed", "photoz_continuity"]#, "photoz_cnst", "photoz_continuity_bursty"]
labels = ['Lognormal', 'Delayed', 'Continuity', 'Constant', 'Continuity Bursty']

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
norm = plt.Normalize( vmin = 4.5, vmax = 10.5)

for marker, bagpipes_run in zip(markers, bagpipes_runs):
    galaxies.comparison_plot(bagpipes_run, "CNST_SFH_RESOLVED_P", label = False, markersize = 4,
                            markeredgecolor = 'black', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'viridis',
                            color_by='redshift', ax = ax, marker = marker, fig = fig, add_colorbar = True if bagpipes_run == "photoz_lognorm" else False, norm = norm) 

# Plot 0.5 dex offset line
xlim = ax.get_xlim()
ylim = ax.get_ylim()

ax.plot(xlim, np.array(ylim)+0.5, color='black', linestyle='--', linewidth=1)
# Label the line with angle text near the top of the line
ax.text(9.35, 9.95, r'0.5 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

ax.plot(xlim, np.array(ylim)+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
ax.text(8.85, 9.95, r'1.0 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

# Fake a legend with the markers
points = []
for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
    point = ax.plot([], [], marker = marker, color = 'black', label = labels[pos])
    points.append(point[0])

ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Bagpipes Run')

ax.set_xlabel(r'Integrated Stellar Mass [$\log_{10}(M_{\odot})$]')
ax.set_ylabel(r'Resolved Stellar Mass [$\log_{10}(M_{\odot})$]')

fig.savefig('../plots/resolved_mass_comparison.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')

## Mass Discrepancy as a Function of sSFR

In [ ]:
# use the table of results this time - loop over each SFH history again, plot sSFR vs delta mass. Propogate errors through the calculation.
#plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')

fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi = 200, tight_layout = True, facecolor = 'white')

plot_dex_diff = True

delta_masses = {}
integrated = {}

colors = [
    "#D14D41",
    "#DA702C",
    #"#D0A215",
    "#879A39",
    "#3AA99F",
    "#4385BE",
    "#8B7EC8",
    "#CE5D97",
]

for marker, bagpipes_run, color in zip(markers, bagpipes_runs, colors):
    if bagpipes_run == 'photoz_dpl':
        # Something weird going on with this run, skip it for now
        continue
    delta_mass = table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1] - table[f'{bagpipes_run}_stellar_mass_50']

    delta_masses[bagpipes_run] = delta_mass
    integrated[bagpipes_run] = table[f'{bagpipes_run}_stellar_mass_50']
    

    upper_error_resolved = table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 2] - table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1]
    lower_error_resolved = table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1] - table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 0]

    upper_error_bagpipes = table[f'{bagpipes_run}_stellar_mass_84'] - table[f'{bagpipes_run}_stellar_mass_50']
    lower_error_bagpipes = table[f'{bagpipes_run}_stellar_mass_50'] - table[f'{bagpipes_run}_stellar_mass_16']

    # Average the errors
    error_resolved = (upper_error_resolved + lower_error_resolved) / 2
    error_bagpipes = (upper_error_bagpipes + lower_error_bagpipes) / 2

    # Propogate errors
    error = np.sqrt(error_resolved**2 + error_bagpipes**2)


    if not plot_dex_diff:
        # Instead plot ratio of resolved mass to bagpipes mass
        delta_mass =  10**table[f'{bagpipes_run}_stellar_mass_50']/10**table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1] 
        error = np.abs(delta_mass) * np.sqrt((error / table[f'{bagpipes_run}_stellar_mass_50'])**2 + (error / table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1])**2)



    yerr = error #placeholder for now

    xerr_lower = table[f'{bagpipes_run}_ssfr_10myr_50'] - table[f'{bagpipes_run}_ssfr_10myr_16']
    xerr_upper = table[f'{bagpipes_run}_ssfr_10myr_84'] - table[f'{bagpipes_run}_ssfr_10myr_50']

    xerr = np.array([xerr_lower, xerr_upper])
    xerr[xerr < 0] = 0

    ax.errorbar(table[f'{bagpipes_run}_ssfr_10myr_50'], delta_mass, yerr = yerr, xerr=xerr, marker='none', linestyle='none', alpha = 0.3, zorder=5, color = color, linewidth = 1)
    ax.scatter(table[f'{bagpipes_run}_ssfr_10myr_50'], delta_mass, marker=marker, s=15, edgecolors='black', linewidth=0.5, zorder=10, color = color)

# Fix the x-axis limits
ax.set_xlim(ax.get_xlim())
ax.set_ylim(ax.get_ylim())

# Make a dummy legend
points = []
for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
    point = ax.plot([], [], marker = marker, mec = 'black', label = labels[pos], color=colors[pos])
    points.append(point[0])



ax.set_xlabel(r'10 Myr Specific Star Formation Rate [$\log_{10}(\mathrm{yr}^{-1})$]', fontsize=11)
if plot_dex_diff:
    ax.set_ylabel(r'Stellar Mass Offset [$\log_{10}(M_{\odot})$]', fontsize=11)
else:
    ax.set_ylabel(r'Integrated / Resolved Stellar Mass')
# Plot 1:1, 0.5 and 1 dex lines
xlim = ax.get_xlim()


ax.plot(xlim, [0, 0], color='black', linestyle='--', linewidth=1)

if plot_dex_diff:
    ax.plot(xlim, np.array([0, 0])+0.5, color='black', linestyle='--', linewidth=1)
    #ax.text(-9.9, 0.62, r'0.5 dex', rotation=0, fontsize=8, color='black', ha = 'left', va = 'top')

    ax.plot(xlim, np.array([0, 0])+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
    #ax.text(-9.9, 1.12, r'1.0 dex', rotation=0, fontsize=8, color='black', ha = 'left', va = 'top')
else:
    # Plot 1:1 line
    ax.plot(xlim, [1, 1], color='black', linestyle='--', linewidth=1)
    #ax.text(-9.9, 1, r'1:1', rotation=0, fontsize=8, color='black', ha = 'left', va = 'bottom')


if not plot_dex_diff:
    # log scale for y-axis
    ax.set_ylim(0.001, 2)

    #ax.set_yscale('log')

ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Bagpipes Run', frameon = True, title_fontsize = 8, markerscale = 0.7, labelspacing=0.1)
ax.set_xlim(-10, -6.9)
ax.set_ylim(-0.6, 1.5)

if plot_dex_diff:
    fig.savefig('../plots/ssfr_vs_mass_offset.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')



In [ ]:
# select photoz_lognorm which have delta_masses > 0.2 dex

select_pos = np.zeros(len(galaxies), dtype=bool)
select_pos2 = np.zeros(len(galaxies), dtype=bool)


# Select outshined galaxies

for key in delta_masses.keys():
    select_pos = select_pos | (delta_masses[key] > 0.2)
    select_pos2 = select_pos2 | ((integrated[key] < 8) & (delta_masses[key] < 0.2))

print(np.sum(select_pos))
print(np.sum(select_pos2))

select_pos2 = select_pos2 & ~select_pos
# remove duplicates from select_pos2


galaxy_ids = table['galaxy_id'][select_pos]
galaxy_ids_notoutshined = table['galaxy_id'][select_pos2]

bagpipes_runs = ["photoz_lognorm", "photoz_delayed", "photoz_continuity"]

galaxies_outshined = ResolvedGalaxies([galaxy for galaxy in galaxies if galaxy.galaxy_id in galaxy_ids])

galaxies_not_outshined = ResolvedGalaxies([galaxy for galaxy in galaxies if galaxy.galaxy_id in galaxy_ids_notoutshined])

#galaxies_notoutshined = ##
other_galaxies = ResolvedGalaxies([galaxy for galaxy in galaxies if galaxy.galaxy_id not in galaxy_ids_notoutshined and galaxy.galaxy_id not in galaxy_ids])


In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_lognorm", "photoz_delayed", "photoz_continuity", "photoz_cnst", "photoz_continuity_bursty"]
labels = ['Lognormal', 'Delayed', 'Continuity', 'Constant', 'Continuity Bursty']

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
norm = plt.Normalize( vmin = 0.1, vmax = 12)

for marker, bagpipes_run in zip(markers, bagpipes_runs):
    galaxies_not_outshined.comparison_plot(bagpipes_run, "BURSTY_SFH_RESOLVED_NOMIN", label = False, markersize = 4,
                            markeredgecolor = 'black', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'cmr.guppy',
                            color_by='int_burstiness', ax = ax, marker = marker, fig = fig, add_colorbar = True if bagpipes_run == "photoz_lognorm" else False, norm = norm) 
    try:
        galaxies_outshined.comparison_plot(bagpipes_run, "BURSTY_SFH_RESOLVED_NOMIN", label = False, markersize = 4,
                                markeredgecolor = 'deeppink', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'cmr.guppy',
                                color_by='int_burstiness', ax = ax, marker = marker, fig = fig, add_colorbar = False, norm = norm) 
    except KeyError:
        pass

# Plot 0.5 dex offset line
xlim = ax.get_xlim()
ylim = ax.get_ylim()

cax = fig.get_axes()[1]
cax.set_xlabel('Burstiness')

ax.plot(xlim, np.array(ylim)+0.5, color='black', linestyle='--', linewidth=1)
# Label the line with angle text near the top of the line
ax.text(9.35, 9.95, r'0.5 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

ax.plot(xlim, np.array(ylim)+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
ax.text(8.85, 9.95, r'1.0 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

# Fake a legend with the markers
points = []
for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
    point = ax.plot([], [], marker = marker, color = 'black', label = labels[pos])
    points.append(point[0])

leg = ax.legend(handles = points, loc = 'lower right', fontsize = 8, title = '   Integrated\nBagpipes Run')

# another fake legend to show outline color shows outshined vs not outshined

points = []
point = ax.plot([], [], marker = 'o', color = 'white', label = 'Outshined', markeredgecolor = 'deeppink', markeredgewidth = 0.5)
points.append(point[0])
point = ax.plot([], [], marker = 'o', color = 'white', label = 'Not Outshined', markeredgecolor = 'black', markeredgewidth = 0.5)
points.append(point[0])

ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Galaxy Type')

ax.add_artist(leg)


ax.set_xlabel(r'Integrated Stellar Mass [$\log_{10}(M_{\odot})$]', fontsize=11)
ax.set_ylabel(r'Resolved Stellar Mass (Voronoi Binning) [$\log_{10}(M_{\odot})$]', fontsize=11)


fig.savefig('../plots/resolved_mass_comparison_voronoi_bursty.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')

In [ ]:
map = 'voronoi'
nbins = 0
non_1 = 0
print(len(other_galaxies))
for galaxy in other_galaxies:
    try:
        bins = galaxy.get_number_of_bins(binmap_type = map)
        if bins > 1:
            non_1 += 1
    except TypeError:
        continue
   

    nbins += bins
    
print(nbins, non_1)


In [ ]:
failed = 0 
for galaxy in galaxies:
    if len(np.unique(galaxy.gal_region['detection'])) == 1:
        galaxy.add_det_galaxy_region(overwrite=True)
    try:
        if np.ndim(galaxy.voronoi_map) == 1:
            print(f'Rerunning galaxy {galaxy.galaxy_id}')
            galaxy.voronoi_binning(SNR_reqs=7, galaxy_region='detection', overwrite=True,
                                use_only_widebands=False, plot=True, quiet=False,
                                ref_band = 'combined_average')
        
            galaxy.measure_flux_in_bins(binmap_type='voronoi')

    except (IndexError, ValueError) as e:
        print(e)
        import traceback
        traceback.print_exc()
        print(f"Failed for galaxy {galaxy.galaxy_id}")
        failed += 1

print(f'Failed for {failed} galaxies')

In [ ]:
nrows = len(galaxies_outshined)
ncols = 4

print(nrows)
positions_to_plot = [1, 4, 5, 9, 10, 19, 26, 27]
nrows = len(positions_to_plot)

# Calculate optimal figure size given the number of rows and columns, and the fact that all axis are square

figsize = (ncols * 1.5, nrows * 1.5)

# import make_axes_locatable
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, dpi = 200, tight_layout = True, facecolor = 'white', sharex=False, sharey=False)
# disable axis tick labels

print(np.shape(axes))
for ax in axes.flatten():
    ax.axis('off')

# Label columns
for i, label in enumerate(['RGB', 'piXedfit', 'piXedfit (no min)', 'Voronoi']):
    axes[0, i].text(0.5, 1.05, label, transform=axes[0, i].transAxes, ha='center', va='bottom', fontsize=10)
     

for i, galaxy in enumerate(galaxies_outshined):

    if i not in positions_to_plot:
        continue
    else:
        i = positions_to_plot.index(i)

    ax = axes[i, 0]
    rgb_stretch = 5e-4
    rgb_q = 5
    galaxy.plot_lupton_rgb(ax=ax, fig=fig,show=False, 
            red=["F444W"],
            green=["F356W"],
            blue=["F200W"],
            return_array=False,
            stretch=rgb_stretch,
            q=rgb_q,
            label_bands=True,
            add_compass=True if i == 0 else False,
            add_scalebar=True,
            compass_center = (17, 25),
            text_fontsize='x-small',
        )


    ax = axes[i, 1]
    #cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
    


    map = galaxy.pixedfit_map
    print(galaxy.meta_properties["zbest_fsps_larson_zfree"], galaxy.meta_properties["zbest_16_fsps_larson_zfree"], galaxy.meta_properties["zbest_84_fsps_larson_zfree"])
    mappable = ax.imshow(map, origin='lower', cmap='nipy_spectral_r', vmin=np.min(map), vmax=np.max(map))
    ax.text(0.05, 0.05, f'{galaxy.galaxy_id}\n z={galaxy.redshift:.2f}$^{{+{galaxy.meta_properties["zbest_84_fsps_larson_zfree"][0]-galaxy.meta_properties["zbest_fsps_larson_zfree"]:.2f}}}_{{-{galaxy.meta_properties["zbest_fsps_larson_zfree"]-galaxy.meta_properties["zbest_16_fsps_larson_zfree"][0]:.2f}}}$', transform=ax.transAxes, ha='left', va='bottom', fontsize=8,
              path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.pixedfit_map))-1}  ', 
            transform=ax.transAxes, ha='left', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    #cbar = fig.colorbar(mappable, cax=cbar_ax)

    ax = axes[i, 2]
    #cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
    minmap = galaxy.pixedfit_nomin_map
    mappable = ax.imshow(minmap, origin='lower', cmap='nipy_spectral_r', vmin=np.min(minmap), vmax=np.max(minmap))
    # Label with ID and number of bins
    ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.pixedfit_nomin_map))}', transform=ax.transAxes, ha='left', va='top', fontsize=8,
            path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    #cbar = fig.colorbar(mappable, cax=cbar_ax)

    ax = axes[i, 3]

    try:
        #cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
        voronoi_map = galaxy.voronoi_map
        mappable = ax.imshow(voronoi_map, origin='lower', cmap='nipy_spectral_r', vmin=np.min(voronoi_map), vmax=np.max(voronoi_map))
        # Label with ID and number of bins
        ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.voronoi_map))}', transform=ax.transAxes, ha='left', va='top', fontsize=8,
                path_effects=[pe.withStroke(linewidth=1, foreground='white')])
        #cbar = fig.colorbar(mappable, cax=cbar_ax)
    except AttributeError:
        # remove the axis
        ax.axis('off')

plt.subplots_adjust(wspace=-0.2)

plt.show()  


In [ ]:
# mass comparison for galaxies_outshined

fig, ax = plt.subplots(1, 1, figsize=(7, 7), dpi = 200, tight_layout = True, facecolor = 'white')
ax.set_title('Outshining')
galaxies_outshined.comparison_plot('CNST_SFH_RESOLVED_P', "CNST_SFH_RESOLVED_NOMIN", label = False, markersize = 4, 
                            markeredgecolor = 'black', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'viridis',
                            color_by='redshift', ax = ax, marker = 's', fig = fig, add_colorbar = True, norm = norm) 

fig, ax = plt.subplots(1, 1, figsize=(7, 7), dpi = 200, tight_layout = True, facecolor = 'white')
ax.set_title('No Outshining')
galaxies_not_outshined.comparison_plot('CNST_SFH_RESOLVED_P', "CNST_SFH_RESOLVED_NOMIN", label = False, markersize = 4, 
                            markeredgecolor = 'black', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'viridis',
                            color_by='redshift', ax = ax, marker = 'o', fig = fig, add_colorbar = True, norm = norm) 




In [ ]:
for galaxy in galaxies_not_outshined:
    print(galaxy.galaxy_id)
    print(galaxy.sed_fitting_table['bagpipes']['CNST_SFH_RESOLVED_NOMIN'])


In [ ]:
run_dir = '/nvme/scratch/work/tharvey/EXPANSE/pipes/'
for galaxy in galaxies_outshined:
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), dpi = 200, tight_layout = True, facecolor = 'white')
    fig, _ = galaxy.plot_bagpipes_sfh(run_name = 'CNST_SFH_RESOLVED_P', bins_to_show=['RESOLVED'], run_dir=run_dir, fig=fig, axes=ax)
    ax = fig.get_axes()[0]
    
    fig, _ = galaxy.plot_bagpipes_sfh(run_name = 'photoz_delayed', bins_to_show=['TOTAL_BIN'], run_dir=run_dir, fig=fig, axes=ax)
    #print(ax)
    ax.set_xlim(0, 0.1)
    ax.set_xlabel('Lookback Time [Gyr]')
    ax.set_ylabel('SFR [M$_{\odot}$ yr$^{-1}$]')
    
    plt.show()

In [ ]:
for galaxy in other_galaxies:
    galaxy.pixedfit_processing(gal_region_use="detection", overwrite=True)
    galaxy.pixedfit_binning(save_out = True, SNR_reqs=4, Dmin_bin=1, redc_chi2_limit=100.0, del_r=1.0, overwrite=False, name_out="pixedfit_nomin")
    

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
tot_bins = 0
for galaxy in galaxies_outshined:
    nbins = galaxy.get_number_of_bins()
    new_nbins = len(np.unique(galaxy.pixedfit_nomin_map)) - 1
    plt.scatter(nbins, new_nbins)
    tot_bins += new_nbins
    '''
    figg, axx = plt.subplots(2, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
    # Plot old and new bin maps side by side - make colormap using nipy_spectral_r with different colors for each integer bin
    
    cmap = plt.cm.nipy_spectral_r
    cmap.set_under('white')
    cmap.set_bad('white')
    # Plot old bin map
    axx[0].imshow(galaxy.pixedfit_map, cmap=cmap, vmin=0, vmax=nbins)
    axx[0].set_title('Old Bin Map')
    # Plot new bin map
    axx[1].imshow(galaxy.pixedfit_nomin_map, cmap=cmap, vmin=0, vmax=new_nbins)
    axx[1].set_title('New Bin Map')

    plt.show()
    '''
    #galaxy.pixedfit_plot_binmap()

plt.plot([0, 40], [0, 40], color='black', linestyle='--', linewidth=1)

ax.set_xlabel('Old Bin Number')
ax.set_ylabel('New Bin Number')
print(f'Old total_bins {galaxies_outshined.total_number_of_bins()}')
print(f'Total bins: {tot_bins}')




In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
masses = []
bins = []
for galaxy in galaxies_outshined:
    old_mass = galaxy.get_total_resolved_property('CNST_SFH_RESOLVED_P', 'stellar_mass')
    new_mass = galaxy.get_total_resolved_property('CNST_SFH_RESOLVED_NOMIN', 'stellar_mass')

    delta_mass = new_mass[1] - old_mass[1]
    masses.append(delta_mass)

    nbins_old = galaxy.get_number_of_bins()
    nbins_new = galaxy.get_number_of_bins('pixedfit_nomin')
    bins.append(nbins_new-nbins_old)


ax.scatter(bins, masses)
ax.set_xscale('log')



In [ ]:
galaxies_outshined.comparison_plot("CNST_SFH_RESOLVED_P", "CNST_SFH_RESOLVED_NOMIN", aperture_label='RESOLVED', color_by='redshift')
galaxies_outshined.comparison_plot("CNST_SFH_RESOLVED_P", "BURSTY_SFH_RESOLVED_NOMIN", aperture_label='RESOLVED', color_by='redshift')

In [ ]:
galaxies_outshined.comparison_plot("photoz_delayed", "CNST_SFH_RESOLVED_NOMIN", color_by='redshift')
galaxies_outshined.comparison_plot("photoz_delayed", "BURSTY_SFH_RESOLVED_NOMIN", color_by='redshift')

